In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
from time import sleep
from os import listdir

In [ ]:
# load environment variables (API keys)
load_dotenv()

client = OpenAI()


In [ ]:
# Uplaod file should only be done once


document_path = "documents"

for document in listdir(document_path):
    print("Uploading file: ", document)
    file = client.files.create(
        file=open(f"{document_path}/{document}", "rb"),
        purpose='assistants'
    )

    # append file id to new file
    print("Writing file id to file_ids.txt")
    with open("file_ids.txt", "a") as f:
        f.write(file.id + "\n")



In [ ]:
files = []

for line in open("file_ids.txt", "r").readlines():
    files.append(line.strip())

print("Files: ", files)


In [ ]:

'''
This is the assistant that will be used to answer questions about the book.

the assistant is only create once and can be used multiple times. with assitant.id
'''
assistant = client.beta.assistants.create(
    name="Islam quiz creator",
    instructions="""
    You are a quizz creator for creating quizzes to students as study material.
    Do not give paragraphs but instead only questions.
    Create a quiz about Islam from the provided text.
    Use only the provided text and craete a generall quizz to help people learn about Islam.
    If the answer cannot be found in the articles, write 'I could not find an answer from the provded documents.'""",
    #
    # retrieval tool is used to implement a RAG-system
    # 
    tools=[{"type": "retrieval"}],
    model="gpt-3.5-turbo-1106",
    file_ids=files
)

print(assistant.id)

In [ ]:

# view all assistants

my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)

for assistangt in my_assistants.data:
    print(assistangt)





In [ ]:

# find more on how assistants work here
# https://platform.openai.com/docs/assistants/how-it-works/objects

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Give me 5 quizz questions about Islam. from the provided text only."
)

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistantID,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)


while run.status != "completed":
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    sleep(1)

print(run.status)



In [ ]:

# retrieve and format mesasge 
messages = client.beta.threads.messages.list(thread_id=thread.id)
message = messages.data[0]

# Extract the message content
message_content = message.content[0].text
annotations = message_content.annotations
citations = []

# Iterate over the annotations and add footnotes
for index, annotation in enumerate(annotations):
    # Replace the text with a footnote
    message_content.value = message_content.value.replace(
        annotation.text, f' [{index}]')

    # Gather citations based on annotation attributes
    if (file_citation := getattr(annotation, 'file_citation', None)):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(
            f'[{index}] {file_citation.quote} from {cited_file.filename}')
    elif (file_path := getattr(annotation, 'file_path', None)):
        cited_file = client.files.retrieve(file_path.file_id)
        citations.append(
            f'[{index}] Click <here> to download {cited_file.filename}')
        # Note: File download functionality not implemented above for brevity

# Add footnotes to the end of the message before displaying to user
message_content.value += '\n' + '\n'.join(citations)


In [ ]:
print(message_content.value)